# Filtrar Estaciones AEMET

Filtrar aquellas estaciones Meteorologicas de la AEMET que complan con los siguientes requisitos minimos:

* Tengan Al menos el > 90% de los datos meteorologicos a excepcion de la variable "sol" y las variables horarias

* Sea la mas cerca con los requisitos anteriores a la estacion de calidad del aire correspondiente

In [1]:
setwd("~/Repositories/AirQualityCOVID/")

## Estaciones Meteo

In [2]:
info.sites <- read.csv("data/Curation/AEMET/info_sites_AEMET.csv",
                       stringsAsFactor=FALSE)
head(info.sites)

,site,start_dt,end_dt,miss_dy,fecha,indicativo,nombre,provincia,altitud,tmed,⋯,tmax,horatmax,dir,velmedia,racha,horaracha,presMax,horaPresMax,presMin,horaPresMin
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0149X,2013-01-01,2020-12-31,42,0.9856312,0.9856312,0.9856312,0.9856312,0.9856312,0.9770783,⋯,0.9770783,0.9856312,0.9322614,0.9404721,0.9322614,0.9856312,0.9784468,0.9784468,0.9784468,0.9784468
2,0200E,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.0000000,1.0000000,0.9993158,1.0000000,0.9993158,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000
3,1059X,2013-01-01,2020-12-31,43,0.9852891,0.9852891,0.9852891,0.9852891,0.9852891,0.9476565,⋯,0.9476565,0.9852891,0.9722887,0.9774205,0.9722887,0.9852891,0.9360246,0.9360246,0.9360246,0.9360246
4,1078I,2013-01-01,2020-12-31,507,0.8265481,0.8265481,0.8265481,0.8265481,0.8265481,0.8029422,⋯,0.8029422,0.8265481,0.8073897,0.8114950,0.8073897,0.8265481,0.6914129,0.6914129,0.6914129,0.6914129
5,1082,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.0000000,1.0000000,0.9996579,0.9993158,0.9996579,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000
6,3129,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,0.9996579,⋯,0.9996579,1.0000000,0.9989737,1.0000000,0.9989737,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000


In [3]:
var.horarias <- c('sol', 'horatmin', 'horatmax', 'horaracha', 'horaPresMax', 'horaPresMin')

info.sites <- info.sites[, -which(names(info.sites) %in% var.horarias)]

In [4]:
info.sites[,"numVar"] <- apply(info.sites, 1, 
                             function(row) {
                                 length(row[5: length(row)]) == sum(row[5: length(row)] > 0.9,
                                     na.rm=T)
                                 }
                             )

info.sites <- info.sites[-which(info.sites$numVar == FALSE),]

## Estaciones mas Cercanas

In [14]:
sites.AEMET <- read.csv("data/Curation/AEMET/sites_AEMET.csv",
                        stringsAsFactor=FALSE)

new.sites <- sites.AEMET[which(sites.AEMET$indicativo %in% info.sites$site),]

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud,dist,siteAQ
,<dbl>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>
1,43.29806,BIZKAIA,42,1082,BILBAO AEROPUERTO,8025,-2.906389,5.756578,es0110a
2,43.37500,BIZKAIA,90,1059X,PUNTA GALEA,8059,-3.021667,8.807760,es0110a
4,40.41194,MADRID,667,3195,"MADRID, RETIRO",8222,-3.678056,3.195804,es0115a
6,40.37556,MADRID,690,3196,"MADRID, CUATRO VIENTOS",8223,-3.786111,8.268356,es0115a
7,40.41194,MADRID,667,3195,"MADRID, RETIRO",8222,-3.678056,1.137154,es0118a
9,40.37556,MADRID,690,3196,"MADRID, CUATRO VIENTOS",8223,-3.786111,10.182525,es0118a
11,40.41194,MADRID,667,3195,"MADRID, RETIRO",8222,-3.678056,4.417472,es0120a
12,40.46667,MADRID,609,3129,MADRID AEROPUERTO,8221,-3.555556,10.427830,es0120a


In [17]:
checked_sites <- data.frame()

for (st in levels(as.factor(new.sites$siteAQ))) {
    rows <- new.sites[new.sites$siteAQ == st,]
    checked_sites <- rbind(checked_sites,
                           rows[rows$dist == min(rows$dist),])
}

In [18]:
head(checked_sites)

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud,dist,siteAQ
,<dbl>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>
1,43.29806,BIZKAIA,42,1082,BILBAO AEROPUERTO,8025,-2.906389,5.756578,es0110a
4,40.41194,MADRID,667,3195,"MADRID, RETIRO",8222,-3.678056,3.195804,es0115a
7,40.41194,MADRID,667,3195,"MADRID, RETIRO",8222,-3.678056,1.137154,es0118a
11,40.41194,MADRID,667,3195,"MADRID, RETIRO",8222,-3.678056,4.417472,es0120a


# Guardar Datos

```R
write.csv(checked_sites,
          "data/Curation/AEMET/checked_sites_AEMET.csv",
          row.names=F)

```